In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.datasets import load_boston, load_iris
from sklearn.linear_model import Ridge,Lasso,ElasticNet, LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

import mglearn
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['font.family']='Malgun Gothic'
matplotlib.rcParams['axes.unicode_minus'] = False

import warnings
warnings.simplefilter('ignore')

In [2]:
df = pd.read_excel('../data4/hyundaiCar.xlsx', sheet_name='train') # train과 test가 각 시트로 나누어져 있음
df

,가격,년식,종류,연비,마력,토크,연료,하이브리드,배기량,중량,변속기
0,1885,2015,준중형,11.8,172,21.0,가솔린,0,1999,1300,자동
1,2190,2015,준중형,12.3,204,27.0,가솔린,0,1591,1300,자동
2,1135,2015,소형,15.0,100,13.6,가솔린,0,1368,1035,수동
3,1645,2014,소형,14.0,140,17.0,가솔린,0,1591,1090,자동
4,1960,2015,대형,9.6,175,46.0,디젤,0,2497,1990,자동
...,...,...,...,...,...,...,...,...,...,...,...
66,3802,2015,중형,8.5,290,34.8,가솔린,0,3342,1901,자동
67,1270,2012,소형,13.3,108,13.9,가솔린,0,1396,1040,자동
68,2430,2015,준중형,12.8,186,41.0,디젤,0,1995,1665,자동
69,2870,2015,중형,17.7,156,19.3,가솔린,1,1999,1585,자동


- train데이터셋, test데이터셋 가져오기
- 가격이 라벨

In [3]:
train_df = pd.read_excel('../data4/hyundaiCar.xlsx', sheet_name='train')
test_df = pd.read_excel('../data4/hyundaiCar.xlsx', sheet_name='test')

In [4]:
x_train = train_df.iloc[:,1:]
y_train = train_df['가격']

In [5]:
x_train

,년식,종류,연비,마력,토크,연료,하이브리드,배기량,중량,변속기
0,2015,준중형,11.8,172,21.0,가솔린,0,1999,1300,자동
1,2015,준중형,12.3,204,27.0,가솔린,0,1591,1300,자동
2,2015,소형,15.0,100,13.6,가솔린,0,1368,1035,수동
3,2014,소형,14.0,140,17.0,가솔린,0,1591,1090,자동
4,2015,대형,9.6,175,46.0,디젤,0,2497,1990,자동
...,...,...,...,...,...,...,...,...,...,...
66,2015,중형,8.5,290,34.8,가솔린,0,3342,1901,자동
67,2012,소형,13.3,108,13.9,가솔린,0,1396,1040,자동
68,2015,준중형,12.8,186,41.0,디젤,0,1995,1665,자동
69,2015,중형,17.7,156,19.3,가솔린,1,1999,1585,자동


In [6]:
x_test = test_df.iloc[:,1:]
y_test = test_df['가격']

In [7]:
x_train.shape

(71, 10)

In [8]:
x_test.shape

(31, 10)

# <문자열 encoding>

In [9]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   년식      71 non-null     int64  
 1   종류      71 non-null     object 
 2   연비      71 non-null     float64
 3   마력      71 non-null     int64  
 4   토크      71 non-null     float64
 5   연료      71 non-null     object 
 6   하이브리드   71 non-null     int64  
 7   배기량     71 non-null     int64  
 8   중량      71 non-null     int64  
 9   변속기     71 non-null     object 
dtypes: float64(2), int64(5), object(3)
memory usage: 5.7+ KB


## 문자열을 숫자로 바꾸는 여러 가지 방식
### 1. Label Encoding
### 2. OneHot Encoding
### 3. 판다스 dummy
### 4. 시리즈 replace

In [10]:
x_train[:5]

,년식,종류,연비,마력,토크,연료,하이브리드,배기량,중량,변속기
0,2015,준중형,11.8,172,21.0,가솔린,0,1999,1300,자동
1,2015,준중형,12.3,204,27.0,가솔린,0,1591,1300,자동
2,2015,소형,15.0,100,13.6,가솔린,0,1368,1035,수동
3,2014,소형,14.0,140,17.0,가솔린,0,1591,1090,자동
4,2015,대형,9.6,175,46.0,디젤,0,2497,1990,자동


In [11]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [12]:
x_train['종류']

0     준중형
1     준중형
2      소형
3      소형
4      대형
     ... 
66     중형
67     소형
68    준중형
69     중형
70     대형
Name: 종류, Length: 71, dtype: object

In [13]:
# 숫자로 바뀐 ndarray를 반환!

lbl = LabelEncoder()
x_trainLabel = lbl.fit_transform(x_train['종류'])
x_trainLabel

array([2, 2, 1, 1, 0, 3, 3, 1, 3, 1, 2, 3, 2, 0, 1, 0, 0, 0, 3, 0, 0, 3,
       2, 0, 3, 3, 3, 1, 1, 2, 0, 0, 0, 0, 1, 0, 2, 0, 2, 1, 1, 1, 0, 0,
       0, 2, 1, 0, 2, 1, 3, 3, 0, 2, 2, 0, 3, 2, 0, 0, 2, 0, 1, 0, 0, 1,
       3, 1, 2, 3, 0])

In [14]:
# 차례대로 0, 1, 2, 3으로 바꾸어줌

lbl.classes_

array(['대형', '소형', '준중형', '중형'], dtype=object)

In [15]:
# 소형은 1로 변환되었음

lbl.transform(['소형'])

array([1], dtype=int64)

# OneHotEncoder (0과 1로)
### 1000, 0100 과 같이 해당하면 1로 나머지는 0으로

In [16]:
oneH = OneHotEncoder()
x_trainOne = oneH.fit_transform(x_train['종류'].values.reshape(-1,1)) # ndarray로 적어주기
x_trainOne

<71x4 sparse matrix of type '<class 'numpy.float64'>'
	with 71 stored elements in Compressed Sparse Row format>

In [17]:
x_train['종류'] # 시리즈

0     준중형
1     준중형
2      소형
3      소형
4      대형
     ... 
66     중형
67     소형
68    준중형
69     중형
70     대형
Name: 종류, Length: 71, dtype: object

In [18]:
x_train['종류'].values # ndarray

array(['준중형', '준중형', '소형', '소형', '대형', '중형', '중형', '소형', '중형', '소형',
       '준중형', '중형', '준중형', '대형', '소형', '대형', '대형', '대형', '중형', '대형', '대형',
       '중형', '준중형', '대형', '중형', '중형', '중형', '소형', '소형', '준중형', '대형', '대형',
       '대형', '대형', '소형', '대형', '준중형', '대형', '준중형', '소형', '소형', '소형', '대형',
       '대형', '대형', '준중형', '소형', '대형', '준중형', '소형', '중형', '중형', '대형',
       '준중형', '준중형', '대형', '중형', '준중형', '대형', '대형', '준중형', '대형', '소형',
       '대형', '대형', '소형', '중형', '소형', '준중형', '중형', '대형'], dtype=object)

In [19]:
x_trainOne.toarray()

array([[0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],


In [20]:
oneH.categories_

[array(['대형', '소형', '준중형', '중형'], dtype=object)]

# 판다스 dummy

In [21]:
pd.get_dummies( x_train['종류'] )

,대형,소형,준중형,중형
0,0,0,1,0
1,0,0,1,0
2,0,1,0,0
3,0,1,0,0
4,1,0,0,0
...,...,...,...,...
66,0,0,0,1
67,0,1,0,0
68,0,0,1,0
69,0,0,0,1


## # 편리하게 데이터 프레임으로 만들어 줌!

In [22]:
pd.get_dummies( x_train )

,년식,연비,마력,토크,하이브리드,배기량,중량,종류_대형,종류_소형,종류_준중형,종류_중형,연료_LPG,연료_가솔린,연료_디젤,변속기_수동,변속기_자동
0,2015,11.8,172,21.0,0,1999,1300,0,0,1,0,0,1,0,0,1
1,2015,12.3,204,27.0,0,1591,1300,0,0,1,0,0,1,0,0,1
2,2015,15.0,100,13.6,0,1368,1035,0,1,0,0,0,1,0,1,0
3,2014,14.0,140,17.0,0,1591,1090,0,1,0,0,0,1,0,0,1
4,2015,9.6,175,46.0,0,2497,1990,1,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,2015,8.5,290,34.8,0,3342,1901,0,0,0,1,0,1,0,0,1
67,2012,13.3,108,13.9,0,1396,1040,0,1,0,0,0,1,0,0,1
68,2015,12.8,186,41.0,0,1995,1665,0,0,1,0,0,0,1,0,1
69,2015,17.7,156,19.3,1,1999,1585,0,0,0,1,0,1,0,0,1


In [23]:
# 바꾸고 싶은 특정 칼럼을 선택해도 됨!!

pd.get_dummies( x_train, columns=['연료', '변속기'])

,년식,종류,연비,마력,토크,하이브리드,배기량,중량,연료_LPG,연료_가솔린,연료_디젤,변속기_수동,변속기_자동
0,2015,준중형,11.8,172,21.0,0,1999,1300,0,1,0,0,1
1,2015,준중형,12.3,204,27.0,0,1591,1300,0,1,0,0,1
2,2015,소형,15.0,100,13.6,0,1368,1035,0,1,0,1,0
3,2014,소형,14.0,140,17.0,0,1591,1090,0,1,0,0,1
4,2015,대형,9.6,175,46.0,0,2497,1990,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,2015,중형,8.5,290,34.8,0,3342,1901,0,1,0,0,1
67,2012,소형,13.3,108,13.9,0,1396,1040,0,1,0,0,1
68,2015,준중형,12.8,186,41.0,0,1995,1665,0,0,1,0,1
69,2015,중형,17.7,156,19.3,1,1999,1585,0,1,0,0,1


# 시리즈 replace - 직접 지정(0, 1 ~ 말고 내가 원하는 숫자를 주고 싶을 때)

In [24]:
x_train['종류'].replace(['대형', '중형', '준중형', '소형'], [0, 1, 2, 3])

0     2
1     2
2     3
3     3
4     0
     ..
66    1
67    3
68    2
69    1
70    0
Name: 종류, Length: 71, dtype: int64

# make_column_transformer

In [37]:
from sklearn.compose import make_column_transformer

In [38]:
myt = make_column_transformer( (OneHotEncoder(), 
                                ['종류', '연료', '변속기'])) # 튜플로 주게 되어 있음

In [39]:
# myt = make_column_transformer( (LabelEncoder(), 
#                                 ['종류', '연료', '변속기']))

In [40]:
result = myt.fit_transform(x_train)
result

array([[0., 0., 1., 0., 0., 1., 0., 0., 1.],
       [0., 0., 1., 0., 0., 1., 0., 0., 1.],
       [0., 1., 0., 0., 0., 1., 0., 1., 0.],
       [0., 1., 0., 0., 0., 1., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0., 1., 0., 1.],
       [0., 0., 0., 1., 0., 0., 1., 0., 1.],
       [0., 0., 0., 1., 0., 0., 1., 0., 1.],
       [0., 1., 0., 0., 0., 1., 0., 0., 1.],
       [0., 0., 0., 1., 0., 1., 0., 0., 1.],
       [0., 1., 0., 0., 0., 1., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 1., 0., 1.],
       [0., 0., 0., 1., 0., 0., 1., 0., 1.],
       [0., 0., 1., 0., 0., 0., 1., 1., 0.],
       [1., 0., 0., 0., 1., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 1., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0., 1., 1., 0.],
       [1., 0., 0., 0., 0., 0., 1., 1., 0.],
       [1., 0., 0., 0., 0., 1., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0., 1., 0., 1.],
       [1., 0., 0., 0., 0., 1., 0., 0., 1.],
       [1., 0., 0., 0., 1., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 1., 0., 0., 1.],
       [0.

# 학습 - make_pipeline 이용해서 편리하게!

In [41]:
# 혹은 

# model = make_pipeline(myt, Ridge(alpha=1))
# model.fit(x_train, y_train)

In [42]:
model = make_pipeline(myt, StandardScaler(), Ridge()) # 파이프라인이 여러 개면 헷갈리기 때문에 
param_value = {'ridge__alpha' : [0.001, 0.01, 0.1, 1, 2, 3]} # 릿지 파라미터라고 명시 - ridge__alpha라고 적어둬야 함!
gridS = GridSearchCV(model, param_grid=param_value, scoring='r2')
gridS.fit(x_train, y_train)
print(gridS.best_params_ )
print(gridS.best_score_ )

{'ridge__alpha': 3}
0.04351266203692761


In [47]:
# gridS.best_estimator_.predict([[2015,'대형',6.8,159,23,'LPG',0,2359,1935,'수동']])

ValueError: Specifying the columns using strings is only supported for pandas DataFrames

In [51]:
a = [2015,'대형',6.8,159,23,'LPG',0,2359,1935,'수동']
b = pd.DataFrame(a).T
x_train.iloc[0:1]
b.columns = x_train.columns
gridS.best_estimator_.predict(b)

array([1869.6491851])